In [197]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

print("Initalized")

Initalized


Table of Content
- [First Look](#First-Look)
- [Redo](#Redo)
- [Bonus](#Bonus)

# First Look

In [218]:
# Read the data
data = pd.read_csv('anonymized-curriculum-access.txt', sep=" ", header=None, na_values='"-"')
# From the https://ds.codeup.com/anomaly-detection/time-series-anomaly-detection-part-3/ rename columns as needed
data.columns=['date', 'time', 'page_viewed','user_id','cohort_id','ip']
# Combine the date time columns and set to datetime object
data['datetime'] = data['date'] + ' ' + data['time']
data['datetime'] = pd.to_datetime(data.datetime)
#Set the index to the new datetime column and sort it
data = data.set_index('datetime')
data['hour'] = data.index.hour
data.head()

,date,time,page_viewed,user_id,cohort_id,ip,hour
datetime,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,9
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,9
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,9
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,9
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,9


In [204]:
# Create an entropy function
def compute_entropy(series):
    counts = series.value_counts()
    if len(counts)==1:
        ent = 0
    else:
        value, counts = np.unique(series, return_counts=True)
        ent = entropy(counts, base=None)
    return ent

compute_entropy(data['cohort_id'])

3.9027926786187983

In [211]:
# Look at the total null values
data.ip.count()

719459

In [274]:
#create a data frame that gets the number of hits per hour date, the probaility of that, and the probaility of having a hit at a certain hour
hour_hits = pd.DataFrame(data['ip'].resample('H').count())
hour_hits.columns=['hits']
hour_hits['hits_proba'] = hour_hits['hits']/data.ip.count()
hour_hits['hour'] = hour_hits.index.hour
temp = hour_hits[['hits', 'hour']].groupby('hour').sum()
temp.reset_index(inplace=True)
temp.columns=['hour', 'hits_by_hour_total']
hour_hits = hour_hits.reset_index().merge(temp, how="left", on='hour').set_index('datetime')
hour_hits['hits_by_hour_proba'] = hour_hits['hits']/hour_hits['hits_by_hour_total']
hour_hits = hour_hits[['hits', 'hour', 'hits_proba', 'hits_by_hour_proba']]
hour_hits

,hits,hour,hits_proba,hits_by_hour_proba
datetime,,,,
2018-01-26 09:00:00,11,9,0.000015,0.000107
2018-01-26 10:00:00,63,10,0.000088,0.000883
2018-01-26 11:00:00,101,11,0.000140,0.001368
2018-01-26 12:00:00,28,12,0.000039,0.000707
2018-01-26 13:00:00,54,13,0.000075,0.000883
...,...,...,...,...
2020-11-02 12:00:00,106,12,0.000147,0.002677
2020-11-02 13:00:00,531,13,0.000738,0.008683
2020-11-02 14:00:00,217,14,0.000302,0.002994


In [224]:
data.resample('H')['ip'].value_counts()

datetime             ip             
2018-01-26 09:00:00  97.105.19.61        11
2018-01-26 10:00:00  97.105.19.61        61
                     71.137.146.71        2
2018-01-26 11:00:00  97.105.19.61       101
2018-01-26 12:00:00  97.105.19.61        22
                                       ... 
2020-11-02 16:00:00  173.172.202.149      1
                     70.121.134.234       1
                     70.121.201.159       1
                     70.94.150.75         1
                     99.43.143.116        1
Name: ip, Length: 97835, dtype: int64

In [199]:
# Make a df of the total views per page
page_views = pd.DataFrame(data['page_viewed'].value_counts()).reset_index()
page_views.columns=['page', 'total_page_views']

# Merge the page views on the original data frame
data = data.merge(page_views, how='left', left_on='page_viewed', right_on='page')
# Drop the page_viewed columns
data.drop(columns=['page_viewed'])
data

,date,time,page_viewed,user_id,cohort_id,ip,datetime,page,total_page_views
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2018-01-26 09:55:03,/,40122.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2018-01-26 09:56:02,java-ii,10374.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2018-01-26 09:56:05,java-ii/object-oriented-programming,5241.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2018-01-26 09:56:06,slides/object_oriented_programming,487.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-26 09:56:24,javascript-i/conditionals,5184.0
...,...,...,...,...,...,...,...,...,...
719454,2020-11-02,16:48:13,javascript-i/coffee-project,763,62.0,107.192.148.199,2020-11-02 16:48:13,javascript-i/coffee-project,2895.0
719455,2020-11-02,16:48:17,javascript-i/mapbox-api,771,62.0,172.125.226.175,2020-11-02 16:48:17,javascript-i/mapbox-api,2567.0
719456,2020-11-02,16:48:18,javascript-i/coffee-project,771,62.0,172.125.226.175,2020-11-02 16:48:18,javascript-i/coffee-project,2895.0
719457,2020-11-02,16:48:28,javascript-i/bom-and-dom/bom,771,62.0,172.125.226.175,2020-11-02 16:48:28,javascript-i/bom-and-dom/bom,3516.0


In [187]:
# Create a dataframe that is grouped by user_id and cohort_id that that counts the number of times each page was viewed by the pair
user_viewed = pd.DataFrame(data[['user_id', 'cohort_id', 'page_viewed']].groupby(['user_id', 'cohort_id'])['page_viewed'].value_counts())
user_viewed.columns=['times_viewed']
user_viewed.reset_index(inplace=True)

#create a dataframe that gets the number of times a page was viewed total per cohort
total_viewed = user_viewed[['cohort_id', 'page_viewed', 'times_viewed']].groupby(['cohort_id', 'page_viewed']).sum().reset_index()
total_viewed.columns=['cohort_id', 'page_viewed', 'total_cohort_views']
user_viewed = user_viewed.merge(total_viewed, on=['cohort_id', 'page_viewed'], how='left') 
# Create the percent viewed 
user_viewed['percent_viewed'] = user_viewed['times_viewed']/user_viewed['total_cohort_views']
user_viewed.sort_values(by=['cohort_id','user_id'])

,user_id,cohort_id,page_viewed,times_viewed,total_cohort_views,percent_viewed
2458,11,1.0,/,237,622,0.381029
2459,11,1.0,javascript-i,150,294,0.510204
2460,11,1.0,html-css,104,215,0.483721
2461,11,1.0,java-ii,85,167,0.508982
2462,11,1.0,javascript-ii,84,204,0.411765
...,...,...,...,...,...,...
89119,776,62.0,mysql/intellij,1,2,0.500000
89120,776,62.0,mysql/sample-database,1,1,1.000000
89121,776,62.0,search/search_index.json,1,121,0.008264
89122,776,62.0,slides/introduction_to_java,1,1,1.000000


In [161]:
# Create a data frame with that groups by user_id, cohort_id that counts the number of total visits for the pair
hits = user_viewed[['user_id','cohort_id']].groupby(['user_id','cohort_id']).size().reset_index()
hits.columns=['user_id', 'cohort_id', 'hits']
# Create a dataframe that sums up the number of hits per cohort
total_hits = hits[['hits','cohort_id']].groupby('cohort_id').sum().reset_index()
total_hits.columns=['cohort_id', 'total_hits_for_cohort']
#merge the two data frames
hits = hits.merge(total_hits, on='cohort_id', how='left')
hits

,user_id,cohort_id,hits,total_hits_for_cohort
0,1,8.0,248,309
1,1,28.0,774,5576
2,2,22.0,151,4811
3,3,22.0,169,4811
4,4,22.0,143,4811
...,...,...,...,...
743,783,59.0,13,3314
744,784,61.0,33,2416
745,785,59.0,14,3314
746,786,56.0,6,3178


# Redo
Needed a fresh look at the data and attemp to gain insights

In [280]:
# Read the data
data = pd.read_csv('anonymized-curriculum-access.txt', sep=" ", header=None, na_values='"-"')
# From the https://ds.codeup.com/anomaly-detection/time-series-anomaly-detection-part-3/ rename columns as needed
data.columns=['date', 'time', 'page_viewed','user_id','cohort_id','ip']
# Combine the date time columns and set to datetime object
data['datetime'] = data['date'] + ' ' + data['time']
data['datetime'] = pd.to_datetime(data.datetime)
#Set the index to the new datetime column and sort it
data = data.set_index('datetime')
# Create columns for hour, day of week, and month
data['hour'] = data.index.hour
data['day'] = data.index.day_name()
data['month'] = data.index.month
# Print the head
data.head()

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month
datetime,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,9,Friday,1
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,9,Friday,1
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,9,Friday,1
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,9,Friday,1
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,9,Friday,1


In [281]:
# Check for percentage of null values
data.isnull().mean()

date           0.000000
time           0.000000
page_viewed    0.000001
user_id        0.000000
cohort_id      0.062325
ip             0.000000
hour           0.000000
day            0.000000
month          0.000000
dtype: float64

In [306]:
# Create a data frame to see the probaility for hits by hour of the day
hour_probaility = pd.DataFrame(data['hour'].value_counts()/data.shape[0]).reset_index()
hour_probaility.columns=['hour', 'probaility']
hour_probaility.sort_values(by='hour')
#store values with a probaility under 1.5% in a list
hours = hour_probaility[hour_probaility['probaility'] < .015]['hour']

15    23
16     7
17     0
18     6
19     1
20     2
21     5
22     4
23     3
Name: hour, dtype: int64

In [311]:
# Probaility of hits happening during the hours listed
data[data.hour.isin(hours)].shape[0]/data.shape[0]

0.048629595293129976

In [351]:
# Create a data frame to see the probaility for page viws 
page_prob = pd.DataFrame(data['page_viewed'].value_counts()/data.shape[0]).reset_index()
page_prob.columns=['page', 'probaility']
page_prob.sort_values(by='probaility',ascending=True)
#store values with a probaility under .00001 in a list
pages = page_prob[page_prob['probaility'] < .00001]['page']
pages

1236                                    11._DistributedML
1237                                   Clustering_Explore
1238                       5-regression/built-in-datasets
1239                              cohorts/24/allcapstones
1240                                        git/branching
                              ...                        
2148    appendix/further-reading/flexbox-additional-co...
2149                                                 .git
2150          appendix/capstone-workbook/project-proposal
2151                               content/git/index.html
2152                                          6.04.05_SVM
Name: page, Length: 917, dtype: object

In [368]:
# Probaility of hits happening during the hours listed
possible = data[(data.page_viewed.isin(pages)) & (data.hour.isin(hours))]
possible[possible['cohort_id'] == 1]

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month
datetime,,,,,,,,,
2018-06-01 23:37:56,2018-06-01,23:37:56,content/javascript/arrays/manipulating.html,209,1.0,177.247.132.187,23,Friday,6
2018-06-01 23:38:33,2018-06-01,23:38:33,content/javascript/arrays/split-join.html,209,1.0,177.247.132.187,23,Friday,6


Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

In [421]:
cohortmin = data[['cohort_id', 'date', 'user_id']].groupby(['cohort_id', 'user_id'])['date'].min().reset_index()
cohortmax = data[['cohort_id', 'date', 'user_id']].groupby(['cohort_id', 'user_id'])['date'].max().reset_index()
cohortmax.columns=['cohort_id', 'user_id', 'max_date']
cohortmin.columns=['cohort_id', 'user_id', 'start_date']

cohort = cohortmin.merge(cohortmax, on=['cohort_id', 'user_id'], how='left')
cohort['max_date'] = pd.to_datetime(cohort.max_date)
cohort['start_date'] = pd.to_datetime(cohort.start_date)
cohort['days_used'] = (cohort['max_date'] - cohort['start_date']).dt.days
cohort[['cohort_id', 'days_used']].groupby(['cohort_id']).mean()

,days_used
cohort_id,
1.0,174.111111
2.0,95.333333
4.0,0.000000
5.0,0.000000
6.0,41.000000
7.0,108.500000
8.0,66.400000
9.0,0.000000
11.0,141.800000


# Bonus

In [109]:
cohort = data[['date', 'cohort_id', 'user_id']].groupby(['cohort_id', 'user_id']).min().reset_index()
cohort

,cohort_id,user_id,date
0,1.0,11,2018-01-26
1,1.0,60,2018-01-29
2,1.0,64,2018-01-29
3,1.0,95,2018-02-10
4,1.0,141,2018-03-14
...,...,...,...
743,62.0,772,2020-09-21
744,62.0,773,2020-09-21
745,62.0,774,2020-09-21
746,62.0,775,2020-09-23
